Centro de Estudos e Sistemas Avançados do Recife

Pós-graduação em Engenharia e Análise de Dados

Disciplina de RNA e Deep Learning

Professor: Silvan Ferreira da Silva Junior 
 
Grupo:
* Anísio Pereira Batista Filho (apbf@cesar.school)
* Carlos Cezar Lopes de Mendonça (cclm@cesar.school)
* Rodolpho Victor França Valsconcelos (rvfv@cesar.school)

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

## Retrieval Augmented Generation (RAG)

### Carregando Documentos - Loading

In [2]:
# https://python.langchain.com/v0.2/docs/how_to/#document-loaders
# https://python.langchain.com/v0.2/docs/integrations/document_loaders/

import bs4
from langchain_community.document_loaders import WebBaseLoader

# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("mw-body-content"))

# Carrega o conteúdo da página
loader = WebBaseLoader(
    web_paths=("https://pt.wikipedia.org/wiki/Centro_de_Estudos_e_Sistemas_Avan%C3%A7ados_do_Recife",),
    bs_kwargs={"parse_only": bs4_strainer},
)

# Carrega o conteúdo da página
docs = loader.load()

len(docs[0].page_content)

USER_AGENT environment variable not set, consider setting it to identify your requests.


6915

In [3]:
print(docs[0].page_content[500:1000])

cife, também conhecido por seu acrônimo CESAR, é um centro de pesquisa e inovação sem fins lucrativos com sede na cidade do Recife, Pernambuco e filiais em Sorocaba, Curitiba e Manaus.[1] O CESAR foi fundado em 1996 por três professores do Centro de Informática da UFPE, Silvio Meira, Fábio Silva e Ismar Kaufman, como forma de aproximar a academia do mercado.[2]
Em 2019, o centro conta com mais de 600 funcionários e em 2018 seu faturamento foi da ordem de R$ 100 milhões.[1][3][4][5]
O CESAR é par


### Dividindo Documentos - Splitting/Chunking

In [4]:
# https://python.langchain.com/v0.2/docs/how_to/#text-splitters

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

10

In [5]:
print(all_splits[2].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Indexando - Store

In [6]:
# https://python.langchain.com/v0.2/docs/how_to/embed_text/

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [7]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Onde está localizado o CESAR?")

len(retrieved_docs)

6

In [8]:
print(retrieved_docs[0].page_content)

Área Educacional[editar | editar código-fonte]
Além de atuar como centro de pesquisa e inovação, o CESAR criou um braço educacional, a CESAR School, oferecendo cursos de graduação, mestrados e doutorados profissionais.[4]
O centro iniciou sua atuação na área educacional em 2007 quando iniciou a oferta do mestrado profissional em Engenharia de Software, que foi avaliado pela CAPES em 2017 como um dos dois melhores mestrados profissionais na área de computação do país.[8] Em 2013 foi autorizada a abertura do segundo mestrado profissional, dessa vez com ênfase em Design de Artefatos Digitais.[9] A partir de 2016 o mestrado em Design passou a também ser oferecido na unidade de Manaus.[10]


### Buscando e Recuperando Informações - Retrieve

In [9]:
from langchain_core.prompts import ChatPromptTemplate

system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [10]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


### Gerando Respostas - Generate

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [13]:
for chunk in rag_chain.stream("Onde fica o CESAR?"):
    print(chunk, end="", flush=True)

O CESAR fica em Recife, Pernambuco. Ele é um centro de pesquisa e inovação com várias filiais em outras cidades brasileiras.

## Exercícios

### Exercício 1
Faça um RAG com um pequeno arquivo de texto, contendo informações que, certamente, a LLM não conheça. Você deverá construir o arquivo e enviar para o ambiente de execução. Escolha a forma de chunking apropriada para o seu documento.

In [14]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

# from google.colab import drive
# drive.mount('/content/drive')

# loader = TextLoader("/content/drive/My Drive/Colab Notebooks/Tópicos Contemporâneos/O Rio Perdido.txt")
loader = TextLoader("data/O Rio Perdido.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=350, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

db = FAISS.from_documents(docs, embeddings)

retriever = db.as_retriever()
llm = OpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

#query = "O que é o Rio Perdido?"
query = "Quais animais e plantas são encontradas no Rio Perdido?"
response = qa_chain.run(query)

print(response)

c:\Users\anisi\anaconda3\envs\cesarschool310\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
c:\Users\anisi\anaconda3\envs\cesarschool310\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


 Animais como o jaguar albino e plantas com flores bioluminescentes são encontrados apenas nessa região.
